<a href="https://colab.research.google.com/github/HaeinGeek/CancerSubtypesAI/blob/main/ipynb/create_gene_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install

In [ ]:
!python -m pip install --user --upgrade pip

In [ ]:
!pip install bioservices
!pip install biopython
!pip install goatools
!pip install pymed
!pip install mygene
!pip install bravado
!pip install torch
!pip install torch-geometric

# Import

In [ ]:
import pandas as pd
import time
import requests
import json
import re
from bioservices import UniProt, KEGG
from Bio import Entrez
from mygene import MyGeneInfo
from bravado.exception import HTTPError
from bravado.client import SwaggerClient
from bravado.requests_client import RequestsClient

# 데이터 로드

In [ ]:
# Load existing dataset
train_df = pd.read_csv('train.csv')
train_df.head()

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TRAIN_0001,SARC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,TRAIN_0002,SKCM,R895R,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TRAIN_0003,KIRC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TRAIN_0004,GBMLGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [ ]:
# Extract gene names
genes = train_df.columns[2:]  # Assuming first two columns are Sample_ID and SUBCLASS

# Remove duplicates
genes = list(set(genes))

# Initialize a DataFrame to store gene information
gene_info_df = pd.DataFrame({'gene': genes})

# 외부 DB 정보 가져오기

In [ ]:
# Initialize services
uniprot = UniProt()
kegg = KEGG()
mg = MyGeneInfo()
Entrez.email = "1995khi@gmail.com"  # Replace with your email

## 함수 정의

In [ ]:
# Define corrected functions
# 1. UniProt
def get_uniprot_info(gene):
    try:
        # Retrieve UniProt ID using NCBI Gene ID
        result = mg.query(gene, scopes='symbol', fields='uniprot', species='human')
        uniprot_info = result['hits'][0].get('uniprot', {})
        uniprot_id = uniprot_info.get('Swiss-Prot') if isinstance(uniprot_info, dict) else None

        # Check if uniprot_id is a list and select the first item if so
        if isinstance(uniprot_id, list):
            uniprot_id = uniprot_id[0]

        # If no UniProt ID is found, try to get protein sequence from NCBI
        if not uniprot_id:
            print(f"No UniProt ID found for gene: {gene}. Trying NCBI...")
            try:
                # Using Entrez to search for protein ID related to the gene name
                result = Entrez.esearch(db="protein", term=f"{gene}[Gene Name] AND Homo sapiens[Organism]")
                record = Entrez.read(result)
                protein_id_list = record['IdList']

                if not protein_id_list:
                    print(f"No protein sequence found in NCBI for gene: {gene}")
                    return {'sequence_length': 0, 'sequence': ''}

                # Fetch the protein sequence using the first protein ID
                protein_id = protein_id_list[0]
                response = Entrez.efetch(db="protein", id=protein_id, rettype="fasta", retmode="text")
                fasta_data = response.read()
                lines = fasta_data.splitlines()
                sequence = ''.join(lines[1:])  # Combine all sequence lines into a single string

                return {
                    'sequence_length': len(sequence),
                    'sequence': sequence
                }

            except Exception as ncbi_error:
                print(f"Failed to retrieve sequence from NCBI for gene {gene}: {ncbi_error}")
                return {'sequence_length': 0, 'sequence': ''}

        # Fetch sequence information from UniProt using the UniProt ID
        fasta_url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.fasta"
        response = requests.get(fasta_url)

        if response.status_code != 200:
            print(f"Failed to retrieve sequence data for UniProt ID: {uniprot_id}")
            return {'sequence_length': 0, 'sequence': ''}

        # Parse the FASTA data
        fasta_data = response.text
        lines = fasta_data.splitlines()
        sequence = ''.join(lines[1:])  # Combine all sequence lines into a single string

        return {
            'sequence_length': len(sequence),
            'sequence': sequence
        }

    except Exception as e:
        print(f"An error occurred while processing gene {gene}: {e}")
        return {'sequence_length': 0, 'sequence': ''}

# 2. STRING
def get_string_info(gene):
    url = "https://string-db.org/api/json/network"

    params = {
        "identifiers": gene,
        "species": 9606,
        "required_score": 400
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        if data:
            max_score = 0
            max_score_partner = ''
            for item in data:
                score = float(item.get('score', 0))
                if score > max_score:
                    max_score = score
                    # Determine the interaction partner gene
                    preferredName_A = item.get('preferredName_A', '')
                    preferredName_B = item.get('preferredName_B', '')
                    if preferredName_A == gene:
                        partner_gene = preferredName_B
                    else:
                        partner_gene = preferredName_A
                    max_score_partner = partner_gene

            return {
                'interaction_count': len(data),
                'highest_score': max_score,
                'highest_score_partner': max_score_partner
            }
        else:
            return {
                'interaction_count': 0,
                'highest_score': 0,
                'highest_score_partner': ''
            }
    except requests.RequestException as e:
        print(f"STRING API call failed for gene {gene}: {e}")
        return {
            'interaction_count': 0,
            'highest_score': 0,
            'highest_score_partner': ''
        }


# # 3. cBioPortal
# 생략


# 4. ClinVar
def get_clinvar_info(gene):
    try:
        handle = Entrez.esearch(db="clinvar", term=f"{gene}[gene]")
        record = Entrez.read(handle)
        variant_count = int(record["Count"])
        time.sleep(0.3)  # To comply with NCBI policies

        handle = Entrez.esearch(db="clinvar", term=f"{gene}[gene] AND pathogenic[Clinical_Significance]")
        record = Entrez.read(handle)
        pathogenic_count = int(record["Count"])
        has_pathogenic = pathogenic_count > 0
        return {
            'variant_count': variant_count,
            'has_pathogenic': has_pathogenic
        }
    except Exception as e:
        print(f"ClinVar data retrieval failed for gene {gene}: {e}")
        return {'variant_count': 0, 'has_pathogenic': False}

# 5. KEGG
def get_kegg_info(gene):
    try:
        pathways = kegg.get_pathway_by_gene(gene, "hsa")
        if pathways:
            return {
                'pathway_count': len(pathways),
                'main_pathway': list(pathways.values())[0] if pathways else 'Unknown'
            }
        return {'pathway_count': 0, 'main_pathway': 'Unknown'}
    except Exception as e:
        print(f"KEGG data retrieval failed for gene {gene}: {e}")
        return {'pathway_count': 0, 'main_pathway': 'Unknown'}

# 6. Gene Ontology
def get_go_info(gene):
    try:
        # Convert gene symbol to NCBI Gene ID
        result = mg.query(gene, scopes='symbol', fields='entrezgene', species='human')
        if result and 'entrezgene' in result['hits'][0]:
            gene_id = result['hits'][0]['entrezgene']
            gene_id = int(gene_id)

            # Gene Ontology API call
            url = f"http://api.geneontology.org/api/bioentity/gene/NCBIGene:{gene_id}/function"
            response = requests.get(url)

            if response.status_code == 200:
                data = response.json()
                associations = data.get('associations', [])

                # Extract all molecular functions and biological processes, removing duplicates using set
                molecular_functions = list(set([assoc['object']['label'] for assoc in associations if 'molecular_activity' in assoc['object']['category']]))
                biological_processes = list(set([assoc['object']['label'] for assoc in associations if 'biological_process' in assoc['object']['category']]))

                return {
                    'molecular_function': molecular_functions if molecular_functions else ['Unknown'],
                    'biological_process': biological_processes if biological_processes else ['Unknown']
                }
        return {'molecular_function': ['Unknown'], 'biological_process': ['Unknown']}

    except Exception as e:
        print(f"Gene Ontology data retrieval failed for gene {gene}: {e}")
        return {'molecular_function': ['Unknown'], 'biological_process': ['Unknown']}


# 7. IntAct
# def get_intact_info(gene):
#     try:
#         # Retrieve UniProt ID using NCBI Gene ID
#         result = mg.query(gene, scopes='symbol', fields='uniprot', species='human')
#         uniprot_info = result['hits'][0].get('uniprot', {})
#         uniprot_id = uniprot_info.get('Swiss-Prot')

#         # Check if uniprot_id is a list and select the first item if so
#         if isinstance(uniprot_id, list):
#             uniprot_id = uniprot_id[0]

#         # If uniprot_id is not found
#         if not uniprot_id:
#             print(f"No UniProt ID found for gene: {gene}")
#             return {'interaction_partners': 0, 'complex_participation': False}

#         # Use UniProt ID for the IntAct API call
#         url = f"https://www.ebi.ac.uk/Tools/webservices/psicquic/intact/webservices/current/search/interactor/{uniprot_id}?format=tab25"
#         response = requests.get(url)

#         if response.status_code == 200:
#             data = response.text.strip().split('\n')
#             interaction_partners = set()
#             complex_participation = False

#             for line in data:
#                 columns = line.split('\t')
#                 interactor_a = columns[0]
#                 interactor_b = columns[1]
#                 interaction_type = columns[11]
#                 interaction_partners.update([interactor_a, interactor_b])

#                 if 'complex' in interaction_type.lower():
#                     complex_participation = True

#             # Exclude the UniProt ID itself
#             interaction_partners = {partner for partner in interaction_partners if uniprot_id not in partner}

#             return {
#                 'interaction_partners': len(interaction_partners),
#                 'complex_participation': complex_participation
#             }

#         return {'interaction_partners': 0, 'complex_participation': False}

#     except Exception as e:
#         print(f"IntAct data retrieval failed for gene {gene}: {e}")
#         return {'interaction_partners': 0, 'complex_participation': False}

In [ ]:
def get_intact_info(gene):
    try:
        # Retrieve UniProt ID and RefSeq Protein ID using NCBI Gene ID
        result = mg.query(gene, scopes='symbol', fields=['uniprot', 'refseq'], species='human')
        uniprot_info = result['hits'][0].get('uniprot', {})
        refseq_info = result['hits'][0].get('refseq', {})
        uniprot_id = uniprot_info.get('Swiss-Prot')

        # Check if uniprot_id is a list and select the first item if so
        if isinstance(uniprot_id, list):
            uniprot_id = uniprot_id[0]

        # If no UniProt ID, try to get RefSeq Protein ID
        if not uniprot_id:
            print(f"No UniProt ID found for gene: {gene}. Trying RefSeq Protein ID...")
            refseq_protein_id = refseq_info.get('protein', [])
            if isinstance(refseq_protein_id, list) and refseq_protein_id:
                uniprot_id = refseq_protein_id[0]  # Use the first RefSeq Protein ID
            else:
                print(f"No RefSeq Protein ID found for gene: {gene}")
                return {'interaction_partners': 0, 'complex_participation': False}

        print(f"Using ID for search: {uniprot_id}")

        # Use UniProt or RefSeq Protein ID for the IntAct API call
        url = f"https://www.ebi.ac.uk/Tools/webservices/psicquic/intact/webservices/current/search/interactor/{uniprot_id}?format=tab25"
        response = requests.get(url)

        if response.status_code == 200:
            data = response.text.strip().split('\n')
            interaction_partners = set()
            complex_participation = False

            for line in data:
                columns = line.split('\t')
                interactor_a = columns[0]
                interactor_b = columns[1]
                interaction_type = columns[11]
                interaction_partners.update([interactor_a, interactor_b])

                if 'complex' in interaction_type.lower():
                    complex_participation = True

            # Exclude the UniProt or RefSeq ID itself
            interaction_partners = {partner for partner in interaction_partners if uniprot_id not in partner}

            return {
                'interaction_partners': len(interaction_partners),
                'complex_participation': complex_participation
            }

        # If the first IntAct API call fails
        print(f"Failed to retrieve data from IntAct API for ID: {uniprot_id}. Trying alternative method...")
        return get_intact_info_via_api(gene)

    except Exception as e:
        # If an exception occurs, attempt to retrieve data using the alternative method
        print(f"IntAct data retrieval failed for gene {gene}: {e}. Trying alternative method...")
        return get_intact_info_via_api(gene)

def get_intact_info_via_api(gene):
    try:
        # Retrieve UniProt ID and RefSeq Protein ID using NCBI Gene ID
        result = mg.query(gene, scopes='symbol', fields=['uniprot', 'refseq'], species='human')
        uniprot_info = result['hits'][0].get('uniprot', {})
        refseq_info = result['hits'][0].get('refseq', {})
        uniprot_id_list = uniprot_info.get('Swiss-Prot')

        # Check if uniprot_id is a list and select the first item if so
        if isinstance(uniprot_id_list, list):
            uniprot_id = uniprot_id_list[0]
        else:
            uniprot_id = uniprot_id_list

        # If no UniProt ID, try to get RefSeq Protein ID
        if not uniprot_id:
            print(f"No UniProt ID found for gene: {gene}. Trying RefSeq Protein ID...")
            refseq_protein_id = refseq_info.get('protein', [])
            if isinstance(refseq_protein_id, list) and refseq_protein_id:
                uniprot_id = refseq_protein_id[0]  # Use the first RefSeq Protein ID
            else:
                print(f"No RefSeq Protein ID found for gene: {gene}")
                return {'interaction_partners': 0, 'complex_participation': False}

        print(f"Using ID for search: {uniprot_id}")

        # Use the Interaction API endpoint
        url = f"https://www.ebi.ac.uk/intact/ws/interaction/findInteractions/{uniprot_id}"
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()  # Parse JSON response
            interaction_partners = set()
            complex_participation = False

            # Extract the 'content' field, which contains interaction details
            content = data.get('content', [])

            # Loop through each interaction entry in 'content'
            for interaction in content:
                # Extract interactor A and B
                interactor_a = interaction.get('intactNameA', '')
                interactor_b = interaction.get('intactNameB', '')

                # Add both interactors to the set of partners
                if interactor_a:
                    interaction_partners.add(interactor_a)
                if interactor_b:
                    interaction_partners.add(interactor_b)

                # Check if the interaction is a complex
                interaction_type = interaction.get('type', '').lower()
                if 'complex' in interaction_type:
                    complex_participation = True

            # Exclude the search ID itself from the partners set
            interaction_partners.discard(uniprot_id)

            return {
                'interaction_partners': len(interaction_partners),
                'complex_participation': complex_participation
            }

        print(f"Failed to retrieve data from IntAct Interaction API for ID: {uniprot_id}. Status code: {response.status_code}")
        return {'interaction_partners': 0, 'complex_participation': False}

    except Exception as e:
        print(f"An error occurred while retrieving IntAct data for ID {uniprot_id}: {e}")
        return {'interaction_partners': 0, 'complex_participation': False}

## 함수 테스트

In [ ]:
gene = 'PTEN'

In [ ]:
get_uniprot_info(gene)

P60484


{'sequence_length': 403,
 'sequence': 'MTAIIKEIVSRNKRRYQEDGFDLDLTYIYPNIIAMGFPAERLEGVYRNNIDDVVRFLDSKHKNHYKIYNLCAERHYDTAKFNCRVAQYPFEDHNPPQLELIKPFCEDLDQWLSEDDNHVAAIHCKAGKGRTGVMICAYLLHRGKFLKAQEALDFYGEVRTRDKKGVTIPSQRRYVYYYSYLLKNHLDYRPVALLFHKMMFETIPMFSGGTCNPQFVVCQLKVKIYSSNSGPTRREDKFMYFEFPQPLPVCGDIKVEFFHKQNKMLKKDKMFHFWVNTFFIPGPEETSEKVENGSLCDQEIDSICSIERADNDKEYLVLTLTKNDLDKANKDKANRYFSPNFKVKLYFTKTVEEPSNPEASSSTSVTPDVSDNEPDHYRYSDTTDSDPENEPFDEDQHTQITKV'}

In [ ]:
get_string_info(gene)

{'interaction_count': 30,
 'highest_score': 0.999,
 'highest_score_partner': 'PIK3CA'}

In [ ]:
get_clinvar_info(gene)

{'variant_count': 3590, 'has_pathogenic': True}

In [ ]:
get_kegg_info(gene)

{'pathway_count': 28, 'main_pathway': 'Inositol phosphate metabolism'}

In [ ]:
get_go_info(gene)

{'molecular_function': ['RNA polymerase II cis-regulatory region sequence-specific DNA binding',
  'TFIID-class transcription factor complex binding',
  'DNA binding',
  'protein binding',
  'DNA-binding transcription factor activity',
  "mRNA 3'-UTR binding",
  'copper ion binding',
  'DNA-binding transcription activator activity, RNA polymerase II-specific',
  'DNA-binding transcription factor activity, RNA polymerase II-specific',
  'cis-regulatory region sequence-specific DNA binding',
  'p53 binding',
  'core promoter sequence-specific DNA binding',
  'DNA-binding transcription repressor activity, RNA polymerase II-specific',
  'transcription cis-regulatory region binding',
  'chromatin binding',
  'protease binding'],
 'biological_process': ['cellular response to glucose starvation',
  'ER overload response',
  'regulation of apoptotic process',
  'regulation of cell cycle',
  'positive regulation of transcription by RNA polymerase II',
  'regulation of cell cycle G2/M phase tran

In [ ]:
get_intact_info(gene)

{'interaction_partners': 37, 'complex_participation': False}

## 데이터 수집

In [ ]:
# Initialize an empty list to store gene data
gene_data_list = []

for gene in genes:
    print(f"Processing gene: {gene}")
    gene_data = {'gene': gene}

    # UniProt
    uniprot_info = get_uniprot_info(gene)
    gene_data.update(uniprot_info)

    # STRING
    string_info = get_string_info(gene)
    gene_data.update(string_info)

    # # cBioPortal
    # cbioportal_info = get_cbioportal_info(gene)
    # gene_data.update(cbioportal_info)

    # ClinVar
    clinvar_info = get_clinvar_info(gene)
    gene_data.update(clinvar_info)

    # KEGG
    kegg_info = get_kegg_info(gene)
    gene_data.update(kegg_info)

    # Gene Ontology
    go_info = get_go_info(gene)
    gene_data.update(go_info)

    # IntAct
    intact_info = get_intact_info(gene)
    gene_data.update(intact_info)

    gene_data_list.append(gene_data)
    time.sleep(0.2)  # To avoid overwhelming the servers

Processing gene: TANK
Processing gene: PPP1CA
Processing gene: PRC1
Processing gene: NDUFAF4
Processing gene: PGM3
Processing gene: LIF
Processing gene: UQCRQ
Processing gene: UPF1
Processing gene: SMC4
Processing gene: DSC3
Processing gene: RAD51
Processing gene: HLA-DRB1
Processing gene: PREX1
Processing gene: ESYT1
Processing gene: ABCC8
Processing gene: EPHB4
Processing gene: PLEK2
Processing gene: PEX5
Processing gene: KLF5
Processing gene: IFIT2
Processing gene: MAPRE3
Processing gene: ITGA9
Processing gene: LMAN1
Processing gene: RALA
Processing gene: UGCG
Processing gene: OPHN1
Processing gene: KDM4B
Processing gene: COL7A1
Processing gene: ANLN
Processing gene: KIT
Processing gene: SLC66A2
Processing gene: LPIN2
Processing gene: PLCL1
Processing gene: CCL25
Processing gene: PRDX3
Processing gene: CDK1
Processing gene: BUB1
Processing gene: NHERF1
Processing gene: LPL
Processing gene: SLC1A1
Processing gene: TTC39B
Processing gene: PIK3CG
Processing gene: UCK2
Processing gene: 

WARNING [bioservices.KEGG:596]:  status is not ok with Not Found
WARNING [bioservices.KEGG:1181]:  Could not parse the entry correctly.


No UniProt ID found for gene: CROCCP2
Processing gene: MLLT10
Processing gene: POLR2G
Processing gene: DCX
Processing gene: BLCAP
Processing gene: GALNT1
Processing gene: HSP90AB1
Processing gene: IDI1
Processing gene: IL2RA
Processing gene: CAB39
Processing gene: JAK2
Processing gene: STEAP3
Processing gene: INS
Processing gene: KIF5B
Processing gene: NSD2
Processing gene: UBE2T
Processing gene: ALDOA
Processing gene: ZBTB10
Processing gene: REG1A
Processing gene: GART
Processing gene: NCBP2
Processing gene: SOWAHC
Processing gene: EBP
Processing gene: GCK
Processing gene: NOC4L
Processing gene: NUFIP1
Processing gene: ACTR2
Processing gene: HPGD
Processing gene: F3
Processing gene: TH
Processing gene: SLC4A4
Processing gene: GYS2
Processing gene: ELOA
Processing gene: ITGB7
Processing gene: PSMC3IP
Processing gene: BCL2L1
Processing gene: RABIF
Processing gene: ATG4A
Processing gene: TNNT3
Processing gene: ZNF292
Processing gene: TCF7L2
Processing gene: WSB1
Processing gene: IFNGR1
P

WARNING [bioservices.KEGG:596]:  status is not ok with Not Found
WARNING [bioservices.KEGG:1181]:  Could not parse the entry correctly.


No UniProt ID found for gene: PVT1
Processing gene: RAC2
Processing gene: TM4SF1
Processing gene: TPD52
Processing gene: VCAM1
Processing gene: E2F3
Processing gene: PLIN2
Processing gene: PPP3R1
Processing gene: PYGL
Processing gene: CCDC28A
Processing gene: CYP2J2
Processing gene: EMP2
Processing gene: SPRY2
Processing gene: ATRX
Processing gene: ZWINT
Processing gene: SLC27A2
Processing gene: C1QC
Processing gene: ATP5F1A
Processing gene: SLC25A20
Processing gene: LY96
Processing gene: HTRA1
Processing gene: RRP12
Processing gene: B4GALT5
Processing gene: HTATIP2
Processing gene: CDKN2C
Processing gene: CALU
Processing gene: MLYCD
Processing gene: PLCG1
Processing gene: TJP1
Processing gene: RPA2
Processing gene: ATP5ME
Processing gene: CNOT6
No UniProt ID found for gene: CNOT6
No UniProt ID found for gene: CNOT6
Processing gene: LCP1
Processing gene: PPP2R5B
Processing gene: TUBGCP3
Processing gene: ANKRA2
Processing gene: TAF12
Processing gene: PLAUR
Processing gene: CAV2
Processi

...

ClinVar data retrieval failed for gene HMGCR: <urlopen error [Errno 11001] getaddrinfo failed>
Gene Ontology data retrieval failed for gene HMGCR: HTTPSConnectionPool(host='mygene.info', port=443): Max retries exceeded with url: /v3/query/?scopes=symbol&fields=entrezgene&species=human&q=HMGCR (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001B337DF12A0>: Failed to resolve 'mygene.info' ([Errno 11001] getaddrinfo failed)"))
IntAct data retrieval failed for gene HMGCR: HTTPSConnectionPool(host='mygene.info', port=443): Max retries exceeded with url: /v3/query/?scopes=symbol&fields=uniprot&species=human&q=HMGCR (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001B338FCCE20>: Failed to resolve 'mygene.info' ([Errno 11001] getaddrinfo failed)"))
Processing gene: CDCA3
An error occurred while processing gene CDCA3: HTTPSConnectionPool(host='mygene.info', port=443): Max retries exceeded with url: /v3/query/?scopes=symbol&fi

CRITICAL[bioservices.KEGG:711]:  HTTPConnectionPool(host='rest.kegg.jp', port=80): Max retries exceeded with url: /get/hsa:CDCA3 (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000001B33A6D3B80>: Failed to resolve 'rest.kegg.jp' ([Errno 11001] getaddrinfo failed)"))
CRITICAL[bioservices.KEGG:712]:  Query unsuccesful. Maybe too slow response.
    Consider increasing it with settings.TIMEOUT attribute 30
WARNING [bioservices.KEGG:1181]:  Could not parse the entry correctly.


ClinVar data retrieval failed for gene CDCA3: <urlopen error [Errno 11001] getaddrinfo failed>
Gene Ontology data retrieval failed for gene CDCA3: HTTPSConnectionPool(host='mygene.info', port=443): Max retries exceeded with url: /v3/query/?scopes=symbol&fields=entrezgene&species=human&q=CDCA3 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001B338FCD180>: Failed to resolve 'mygene.info' ([Errno 11001] getaddrinfo failed)"))
IntAct data retrieval failed for gene CDCA3: HTTPSConnectionPool(host='mygene.info', port=443): Max retries exceeded with url: /v3/query/?scopes=symbol&fields=uniprot&species=human&q=CDCA3 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001B33CE25F00>: Failed to resolve 'mygene.info' ([Errno 11001] getaddrinfo failed)"))
Processing gene: SLC12A4
An error occurred while processing gene SLC12A4: HTTPSConnectionPool(host='mygene.info', port=443): Max retries exceeded with url: /v3/query/?scopes=symbo

CRITICAL[bioservices.KEGG:711]:  HTTPConnectionPool(host='rest.kegg.jp', port=80): Max retries exceeded with url: /get/hsa:SLC12A4 (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000001B33A6D0700>: Failed to resolve 'rest.kegg.jp' ([Errno 11001] getaddrinfo failed)"))
CRITICAL[bioservices.KEGG:712]:  Query unsuccesful. Maybe too slow response.
    Consider increasing it with settings.TIMEOUT attribute 30
WARNING [bioservices.KEGG:1181]:  Could not parse the entry correctly.


ClinVar data retrieval failed for gene SLC12A4: <urlopen error [Errno 11001] getaddrinfo failed>
Gene Ontology data retrieval failed for gene SLC12A4: HTTPSConnectionPool(host='mygene.info', port=443): Max retries exceeded with url: /v3/query/?scopes=symbol&fields=entrezgene&species=human&q=SLC12A4 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001B33A6BD600>: Failed to resolve 'mygene.info' ([Errno 11001] getaddrinfo failed)"))
IntAct data retrieval failed for gene SLC12A4: HTTPSConnectionPool(host='mygene.info', port=443): Max retries exceeded with url: /v3/query/?scopes=symbol&fields=uniprot&species=human&q=SLC12A4 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001B338FEB8E0>: Failed to resolve 'mygene.info' ([Errno 11001] getaddrinfo failed)"))
Processing gene: KRT1
An error occurred while processing gene KRT1: HTTPSConnectionPool(host='mygene.info', port=443): Max retries exceeded with url: /v3/query/?scopes=s

In [ ]:
gene_data_list[:10]

[{'gene': 'TANK',
  'sequence_length': 425,
  'sequence': 'MDKNIGEQLNKAYEAFRQACMDRDSAVKELQQKTENYEQRIREQQEQLSLQQTIIDKLKSQLLLVNSTQDNNYGCVPLLEDSETRKNNLTLDQPQDKVISGIAREKLPKVRRQEVSSPRKETSARSLGSPLLHERGNIEKTFWDLKEEFHKICMLAKAQKDHLSKLNIPDTATETQCSVPIQCTDKTDKQEALFKPQAKDDINRGAPSITSVTPRGLCRDEEDTSFESLSKFNVKFPPMDNDSTFLHSTPERPGILSPATSEAVCQEKFNMEFRDNPGNFVKTEETLFEIQGIDPIASAIQNLKTTDKTKPSNLVNTCIRTTLDRAACLPPGDHNALYVNSFPLLDPSDAPFPSLDSPGKAIRGPQQPIWKPFPNQDSDSVVLSGTDSELHIPRVCEFCQAVFPPSITSRGDFLRHLNSHFNGET',
  'interaction_count': 52,
  'highest_score': 0.999,
  'highest_score_partner': 'TNFRSF1A',
  'variant_count': 51,
  'has_pathogenic': True,
  'pathway_count': 6,
  'main_pathway': 'Autophagy - animal',
  'molecular_function': ['cysteine-type deubiquitinase activity',
   'ubiquitin protein ligase binding',
   'protein binding',
   'metal ion binding',
   'molecular function inhibitor activity',
   'molecular adaptor activity',
   'deubiquitinase activator activity'],
  'biological_process': ['negative regula

In [ ]:
# Create DataFrame from gene data
gene_info_df = pd.DataFrame(gene_data_list)

In [ ]:
# Save to CSV for future use
gene_info_df.to_csv('gene_info.csv', index=False)

## 오류 발생 유전자 처리

In [ ]:
error_genes = pd.read_csv('Gene_Errors_by_Category.csv')
error_genes.head()

,No UniProt ID found,STRING API call failed,IntAct data retrieval failed,Failed to retrieve sequence data
0,ADIG,ACP2,RHD,"['Q86W56', 'Q86WA6']"
1,ADIG,AMD1,CD40LG,"['P58401', 'Q9P2S2']"
2,USP11,CROCCP2,CLPS,"['P42167', 'P42166']"
3,USP11,PVT1,RDH16,"['Q5R372', 'B7ZAP0']"
4,TEX15,H2AC25,GYPE,"['Q9NZJ9', 'A0A024RBG1']"


### 1. UNIPROT ID가 없는 경우

In [ ]:
# 1. uniprot_id 가 없는 경우 ncbi prtein 검색 결과 활용
no_uniprot_id = error_genes.iloc[:,0].dropna().drop_duplicates().values
no_uniprot_id

array(['ADIG', 'USP11', 'TEX15', 'CROCCP2', 'PVT1', 'CNOT6', 'HBBP1',
       'PTTG3P', 'PRPF4B', 'SAC3D1', 'PAX4'], dtype=object)

### 2. STRING API 호출 되지 않는 경우
- string_api_call_failed 의 경우 상호작용 결과가 없는 것이므로 pass

In [ ]:
# 2. string_api_call_failed 의 경우 상호작용 결과가 없는 것이므로 pass
string_api_call_failed = error_genes.iloc[:,1].dropna().drop_duplicates().values
string_api_call_failed

array(['ACP2', 'AMD1', 'CROCCP2', 'PVT1', 'H2AC25', 'TARP', 'HBBP1',
       'OPRK1', 'MDH1', 'PTTG3P', 'SKIC3', 'MIX23', 'GPX1', 'LDHB',
       'DNAJC12', 'CYP26B1', 'MAP1S', 'PTPN1', 'BCL2L11', 'CSNK2A2',
       'RAP1GDS1', 'CYC1', 'TSPO2', 'ZNF112', 'AFG3L2', 'AGR2', 'MAPK9',
       'SLC6A14', 'SIK1', 'IFIT1', 'ICAM1', 'P4HA1', 'EPB41', 'TNFRSF1B',
       'CLN6', 'EXT2', 'SKP1', 'TNFAIP6', 'RHOF', 'CFI', 'DAP', 'ATP5PO',
       'PTX3', 'TKFC', 'SCRN1', 'ATAD2', 'MBL2', 'SMOX', 'FOXC1', 'FBXO7',
       'GPER1', 'GNAO1', 'ABCA3', 'BAIAP2', 'FGD6', 'ARPC5L', 'FH',
       'MEF2C', 'PLA2G12A', 'SESN1', 'THSD4', 'IRF1', 'MAL2', 'AFF1',
       'TAL1', 'SLC25A23', 'CCR1', 'CNN2', 'SPEG', 'THBD', 'PLCB2',
       'SKAP2', 'TP63', 'AKR1C2', 'EIF2AK3', 'KLF3', 'GAMT', 'MRPL34',
       'DIO2', 'FOXO1', 'MYL12B', 'SPTA1', 'ARHGEF2', 'CYB5B', 'DLG5',
       'PDLIM5', 'SMTN', 'TRAF4', 'AMACR', 'CXCL6', 'MTMR10', 'TAT',
       'AQP3', 'ENTPD7', 'ERCC2', 'GLO1', 'PIK3R3', 'RNASEL', 'MYOF',
       'XRC

In [ ]:
len(string_api_call_failed)

384

In [ ]:
gene_info_df[gene_info_df.gene.isin(string_api_call_failed)]

,gene,sequence_length,sequence,interaction_count,highest_score,highest_score_partner,variant_count,has_pathogenic,pathway_count,main_pathway,molecular_function,biological_process,interaction_partners,complex_participation
354,ACP2,423,MAGKRSGWSRAALLQLLLGVNLVVMPPTRARSLRFVTLLYRHGDRS...,0,0.0,NaN,52,True,3,Riboflavin metabolism,"['acid phosphatase activity', 'protein binding']",['lysosome organization'],68,False
423,AMD1,334,MEAAHFFEGTEKLLEVWFSRQQPDANQGSGDLRTIPRSEWDILLKD...,0,0.0,NaN,42,True,3,Cysteine and methionine metabolism,"['putrescine binding', 'adenosylmethionine dec...","['spermidine biosynthetic process', 'spermine ...",46,False
810,CROCCP2,0,NaN,0,0.0,NaN,0,False,0,Unknown,['Unknown'],['Unknown'],0,False
896,PVT1,0,NaN,0,0.0,NaN,55,True,0,Unknown,['Unknown'],['Unknown'],0,False
941,H2AC25,130,MSGRGKQGGKARAKAKSRSSRAGLQFPVGRVHRLLRKGNYSERVGA...,0,0.0,NaN,52,True,5,ATP-dependent chromatin remodeling,"['protein heterodimerization activity', 'prote...","['heterochromatin formation', 'UV-damage excis...",25,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3242,PKP1,0,NaN,0,0.0,NaN,0,False,0,Unknown,['Unknown'],['Unknown'],0,False
3243,PSMA3,0,NaN,0,0.0,NaN,0,False,0,Unknown,['Unknown'],['Unknown'],0,False
3244,ABHD6,0,NaN,0,0.0,NaN,0,False,0,Unknown,['Unknown'],['Unknown'],0,False
3245,BUB3,0,NaN,0,0.0,NaN,0,False,0,Unknown,['Unknown'],['Unknown'],0,False


### 3. IntAct data 수집되지 않는 경우
- 엔드포인트 주소 변경으로 인해 일부 UNIPORT ID로 검색 불가

In [ ]:
# 3.
intact_data_retrieval_failed = error_genes.iloc[:,2].dropna().drop_duplicates().values
intact_data_retrieval_failed = intact_data_retrieval_failed[:-1]
intact_data_retrieval_failed

array(['RHD', 'CD40LG', 'CLPS', 'RDH16', 'GYPE', 'ACKR1', 'BPHL',
       'CLDN16', 'KCNK15', 'CTNS', 'CRTAM', 'ARTN', 'G6PC2', 'NRXN2',
       'HSD11B2', 'TMPO', 'TRIM58', 'CDH6', 'TARP', 'SCUBE2', 'CMPK2',
       'ARPP21', 'PRSS36', 'CAPN9', 'CSF3R', 'RABGAP1L', 'GRID2',
       'PTGER4', 'CXCR6', 'NUDT4', 'IL9', 'MT3', 'CCR7', 'LCT', 'NKX6-1',
       'PAOX', 'AQP7', 'THEG', 'ST6GALNAC2', 'GPR132', 'SELL', 'ITGBL1',
       'ITIH5', 'GPR87', 'GPR19', 'PAPOLB', 'GNAS', 'TAS2R4', 'MYL12A',
       'MOCS2', 'HTR1D', 'TOR1AIP2', 'RHCE', 'CLN8', 'CXCL10', 'SLC46A3'],
      dtype=object)

In [ ]:
for gene in intact_data_retrieval_failed:
    print(f"Processing gene: {gene}")
    result = mg.query(gene, scopes='symbol', fields='uniprot', species='human')
    uniprot_info = result['hits'][0].get('uniprot', {})
    uniprot_id = uniprot_info.get('Swiss-Prot') if isinstance(uniprot_info, dict) else None
    print(uniprot_id)

Processing gene: RHD
Q02161
Processing gene: CD40LG
P29965
Processing gene: CLPS
P04118
Processing gene: RDH16
O75452
Processing gene: GYPE
P15421
Processing gene: ACKR1
Q16570
Processing gene: BPHL
['Q86W56', 'Q86WA6']
Processing gene: CLDN16
Q9Y5I7
Processing gene: KCNK15
Q9H427
Processing gene: CTNS
O60931
Processing gene: CRTAM
O95727
Processing gene: ARTN
Q5T4W7
Processing gene: G6PC2
Q9NQR9
Processing gene: NRXN2
['P58401', 'Q9P2S2']
Processing gene: HSD11B2
P80365
Processing gene: TMPO
['P42167', 'P42166']
Processing gene: TRIM58
Q8NG06
Processing gene: CDH6
P55285
Processing gene: TARP
A2JGV3
Processing gene: SCUBE2
Q9NQ36
Processing gene: CMPK2
Q5EBM0
Processing gene: ARPP21
Q9UBL0
Processing gene: PRSS36
Q5K4E3
Processing gene: CAPN9
O14815
Processing gene: CSF3R
Q99062
Processing gene: RABGAP1L
['Q5R372', 'B7ZAP0']
Processing gene: GRID2
O43424
Processing gene: PTGER4
P35408
Processing gene: CXCR6
O00574
Processing gene: NUDT4
['Q9NZJ9', 'A0A024RBG1']
Processing gene: IL9
P1

In [ ]:
import requests

def get_intact_info(gene):
    try:
        # Retrieve UniProt ID using NCBI Gene ID
        result = mg.query(gene, scopes='symbol', fields='uniprot', species='human')
        uniprot_info = result['hits'][0].get('uniprot', {})
        uniprot_id = uniprot_info.get('Swiss-Prot')

        # Check if uniprot_id is a list and select the first item if so
        if isinstance(uniprot_id, list):
            uniprot_id = uniprot_id[0]

        # If uniprot_id is not found
        if not uniprot_id:
            print(f"No UniProt ID found for gene: {gene}")
            return {'interaction_partners': 0, 'complex_participation': False}

        # Use UniProt ID for the IntAct API call
        url = f"https://www.ebi.ac.uk/Tools/webservices/psicquic/intact/webservices/current/search/interactor/{uniprot_id}?format=tab25"
        response = requests.get(url)

        if response.status_code == 200:
            data = response.text.strip().split('\n')
            interaction_partners = set()
            complex_participation = False

            for line in data:
                columns = line.split('\t')
                interactor_a = columns[0]
                interactor_b = columns[1]
                interaction_type = columns[11]
                interaction_partners.update([interactor_a, interactor_b])

                if 'complex' in interaction_type.lower():
                    complex_participation = True

            # Exclude the UniProt ID itself
            interaction_partners = {partner for partner in interaction_partners if uniprot_id not in partner}

            return {
                'interaction_partners': len(interaction_partners),
                'complex_participation': complex_participation
            }

        # If the first IntAct API call fails
        print(f"Failed to retrieve data from IntAct API for UniProt ID: {uniprot_id}. Trying alternative method...")
        return get_intact_info_via_api(gene)

    except Exception as e:
        # If an exception occurs, attempt to retrieve data using the alternative method
        print(f"IntAct data retrieval failed for gene {gene}: {e}. Trying alternative method...")
        return get_intact_info_via_api(gene)

def get_intact_info_via_api(gene):
    try:
        # Retrieve UniProt ID using NCBI Gene ID
        result = mg.query(gene, scopes='symbol', fields='uniprot', species='human')
        uniprot_info = result['hits'][0].get('uniprot', {})
        uniprot_id_list = uniprot_info.get('Swiss-Prot')

        # Check if uniprot_id is a list and select the first item if so
        if isinstance(uniprot_id_list, list):
            uniprot_id = uniprot_id_list[0]
        else:
            uniprot_id = uniprot_id_list

        if not uniprot_id:
            print(f"No UniProt ID found for gene: {gene}")
            return {'interaction_partners': 0, 'complex_participation': False}

        print(uniprot_id)

        # Use the Interaction API endpoint
        url = f"https://www.ebi.ac.uk/intact/ws/interaction/findInteractions/{uniprot_id}"
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()  # Parse JSON response
            # Initialize variables to store interaction data
            interaction_partners = set()
            complex_participation = False

            # Extract the 'content' field, which contains interaction details
            content = data.get('content', [])

            # Loop through each interaction entry in 'content'
            for interaction in content:
                # Extract interactor A and B
                interactor_a = interaction.get('intactNameA', '')
                interactor_b = interaction.get('intactNameB', '')

                # Add both interactors to the set of partners
                if interactor_a:
                    interaction_partners.add(interactor_a)
                if interactor_b:
                    interaction_partners.add(interactor_b)

                # Check if the interaction is a complex
                interaction_type = interaction.get('type', '').lower()
                if 'complex' in interaction_type:
                    complex_participation = True

            # Exclude the UniProt ID itself from the partners set
            interaction_partners.discard(uniprot_id)
            # print(interaction_partners)

            return {
                'interaction_partners': len(interaction_partners),
                'complex_participation': complex_participation
            }

        print(f"Failed to retrieve data from IntAct Interaction API for UniProt ID: {uniprot_id}. Status code: {response.status_code}")
        return {'interaction_partners': 0, 'complex_participation': False}

    except Exception as e:
        print(f"An error occurred while retrieving IntAct data for UniProt ID {uniprot_id}: {e}")
        return {'interaction_partners': 0, 'complex_participation': False}

In [ ]:
for gene in intact_data_retrieval_failed[:5]:
    print(f"Processing gene: {gene}")
    print(get_intact_info(gene))

Processing gene: RHD
IntAct data retrieval failed for gene RHD: list index out of range. Trying alternative method...
Q02161
{'interaction_partners': 8, 'complex_participation': False}
Processing gene: CD40LG
IntAct data retrieval failed for gene CD40LG: list index out of range. Trying alternative method...
P29965
{'interaction_partners': 0, 'complex_participation': False}
Processing gene: CLPS
IntAct data retrieval failed for gene CLPS: list index out of range. Trying alternative method...
P04118
{'interaction_partners': 0, 'complex_participation': False}
Processing gene: RDH16
IntAct data retrieval failed for gene RDH16: list index out of range. Trying alternative method...
O75452
{'interaction_partners': 0, 'complex_participation': False}
Processing gene: GYPE
IntAct data retrieval failed for gene GYPE: list index out of range. Trying alternative method...
P15421
{'interaction_partners': 0, 'complex_participation': False}


### 4. UNIPROT ID 가 리스트로 존재하는 경우

In [ ]:
multi_uniprot_id = error_genes.iloc[:,3].dropna().drop_duplicates().values
multi_uniprot_id

array(["['Q86W56', 'Q86WA6']", "['P58401', 'Q9P2S2']",
       "['P42167', 'P42166']", "['Q5R372', 'B7ZAP0']",
       "['Q9NZJ9', 'A0A024RBG1']",
       "['Q5JWF2', 'P84996', 'P63092', 'O95467']", "['O14950', 'P19105']",
       "['O96007', 'O96033']", "['Q8NFQ8', 'Q9H496']"], dtype=object)

In [ ]:
multi_uniprot_id_genes = ['BPHL', 'NRXN2', 'TMPO', 'RABGAP1L', 'NUDT4', 'GNAS', 'MYL12A', 'MOCS2', 'TOR1AIP2']

In [ ]:
def get_uniprot_info(gene):
    try:
        # Retrieve UniProt ID using NCBI Gene ID
        result = mg.query(gene, scopes='symbol', fields='uniprot', species='human')
        uniprot_info = result['hits'][0].get('uniprot', {})
        uniprot_id = uniprot_info.get('Swiss-Prot')

        # Check if uniprot_id is a list and select the first item if so
        if isinstance(uniprot_id, list):
            uniprot_id = uniprot_id[0]

        # If uniprot_id is not found
        if not uniprot_id:
            print(f"No UniProt ID found for gene: {gene}")
            return {'sequence_length': 0, 'sequence': ''}

        # Fetch sequence information from UniProt using the UniProt ID
        fasta_url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.fasta"
        response = requests.get(fasta_url)

        if response.status_code != 200:
            print(f"Failed to retrieve sequence data for UniProt ID: {uniprot_id}")
            return {'sequence_length': 0, 'sequence': ''}

        # Parse the FASTA data
        fasta_data = response.text
        lines = fasta_data.splitlines()
        sequence = ''.join(lines[1:])  # Combine all sequence lines into a single string

        return {
            'sequence_length': len(sequence),
            'sequence': sequence
        }

    except Exception as e:
        print(f"An error occurred while processing gene {gene}: {e}")
        return {'sequence_length': 0, 'sequence': ''}


# 2. IntAct
def get_intact_info(gene):
    try:
        # Retrieve UniProt ID using NCBI Gene ID
        result = mg.query(gene, scopes='symbol', fields='uniprot', species='human')
        uniprot_info = result['hits'][0].get('uniprot', {})
        uniprot_id = uniprot_info.get('Swiss-Prot')

        # Check if uniprot_id is a list and select the first item if so
        if isinstance(uniprot_id, list):
            uniprot_id = uniprot_id[0]

        # If uniprot_id is not found
        if not uniprot_id:
            print(f"No UniProt ID found for gene: {gene}")
            return {'interaction_partners': 0, 'complex_participation': False}

        # Use UniProt ID for the IntAct API call
        url = f"https://www.ebi.ac.uk/Tools/webservices/psicquic/intact/webservices/current/search/interactor/{uniprot_id}?format=tab25"
        response = requests.get(url)

        if response.status_code == 200:
            data = response.text.strip().split('\n')
            interaction_partners = set()
            complex_participation = False

            for line in data:
                columns = line.split('\t')
                interactor_a = columns[0]
                interactor_b = columns[1]
                interaction_type = columns[11]
                interaction_partners.update([interactor_a, interactor_b])

                if 'complex' in interaction_type.lower():
                    complex_participation = True

            # Exclude the UniProt ID itself
            interaction_partners = {partner for partner in interaction_partners if uniprot_id not in partner}

            return {
                'interaction_partners': len(interaction_partners),
                'complex_participation': complex_participation
            }

        return {'interaction_partners': 0, 'complex_participation': False}

    except Exception as e:
        print(f"IntAct data retrieval failed for gene {gene}: {e}")
        return {'interaction_partners': 0, 'complex_participation': False}

In [ ]:
for gene in ['PTEN']+multi_uniprot_id_genes:
    print(f"Processing gene: {gene}")
    print(get_uniprot_info(gene))

Processing gene: PTEN
{'sequence_length': 403, 'sequence': 'MTAIIKEIVSRNKRRYQEDGFDLDLTYIYPNIIAMGFPAERLEGVYRNNIDDVVRFLDSKHKNHYKIYNLCAERHYDTAKFNCRVAQYPFEDHNPPQLELIKPFCEDLDQWLSEDDNHVAAIHCKAGKGRTGVMICAYLLHRGKFLKAQEALDFYGEVRTRDKKGVTIPSQRRYVYYYSYLLKNHLDYRPVALLFHKMMFETIPMFSGGTCNPQFVVCQLKVKIYSSNSGPTRREDKFMYFEFPQPLPVCGDIKVEFFHKQNKMLKKDKMFHFWVNTFFIPGPEETSEKVENGSLCDQEIDSICSIERADNDKEYLVLTLTKNDLDKANKDKANRYFSPNFKVKLYFTKTVEEPSNPEASSSTSVTPDVSDNEPDHYRYSDTTDSDPENEPFDEDQHTQITKV'}
Processing gene: BPHL
Failed to retrieve sequence data for UniProt ID: ['Q86W56', 'Q86WA6']
{'sequence_length': 0, 'sequence': ''}
Processing gene: NRXN2
Failed to retrieve sequence data for UniProt ID: ['P58401', 'Q9P2S2']
{'sequence_length': 0, 'sequence': ''}
Processing gene: TMPO
Failed to retrieve sequence data for UniProt ID: ['P42167', 'P42166']
{'sequence_length': 0, 'sequence': ''}
Processing gene: RABGAP1L
Failed to retrieve sequence data for UniProt ID: ['Q5R372', 'B7ZAP0']
{'sequence_length': 0, 'sequence': ''}
Proce

In [ ]:
for gene in ['PTEN']+multi_uniprot_id_genes:
    print(f"Processing gene: {gene}")
    print(get_uniprot_info(gene))

Processing gene: PTEN
{'sequence_length': 403, 'sequence': 'MTAIIKEIVSRNKRRYQEDGFDLDLTYIYPNIIAMGFPAERLEGVYRNNIDDVVRFLDSKHKNHYKIYNLCAERHYDTAKFNCRVAQYPFEDHNPPQLELIKPFCEDLDQWLSEDDNHVAAIHCKAGKGRTGVMICAYLLHRGKFLKAQEALDFYGEVRTRDKKGVTIPSQRRYVYYYSYLLKNHLDYRPVALLFHKMMFETIPMFSGGTCNPQFVVCQLKVKIYSSNSGPTRREDKFMYFEFPQPLPVCGDIKVEFFHKQNKMLKKDKMFHFWVNTFFIPGPEETSEKVENGSLCDQEIDSICSIERADNDKEYLVLTLTKNDLDKANKDKANRYFSPNFKVKLYFTKTVEEPSNPEASSSTSVTPDVSDNEPDHYRYSDTTDSDPENEPFDEDQHTQITKV'}
Processing gene: BPHL
{'sequence_length': 976, 'sequence': 'MNAGPGCEPCTKRPRWGAATTSPAASDARSFPSRQRRVLDPKDAHVQFRVPPSSPACVPGRAGQHRGSATSLVFKQKTITSWMDTKGIKTAESESLDSKENNNTRIESMMSSVQKDNFYQHNVEKLENVSQLSLDKSPTEKSTQYLNQHQTAAMCKWQNEGKHTEQLLESEPQTVTLVPEQFSNANIDRSPQNDDHSDTDSEENRDNQQFLTTVKLANAKQTTEDEQAREAKSHQKCSKSCDPGEDCASCQQDEIDVVPESPLSDVGSEDVGTGPKNDNKLTRQESCLGNSPPFEKESEPESPMDVDNSKNSCQDSEADEETSPGFDEQEDGSSSQTANKPSRFQARDADIEFRKRYSTKGGEVRLHFQFEGGESRTGMNDLNAKLPGNISSLNVECRNSKQHGKKDSKITDHFMRLPKAEDRRKEQWETKHQRTERKIPKYVPPHLSPDKKWLGTPIEEMRRMPRCGIRLPLL

In [ ]:
for gene in multi_uniprot_id_genes:
    print(f"Processing gene: {gene}")
    print(get_intact_info(gene))

Processing gene: BPHL
{'interaction_partners': 3, 'complex_participation': False}
Processing gene: NRXN2
IntAct data retrieval failed for gene NRXN2: list index out of range
{'interaction_partners': 0, 'complex_participation': False}
Processing gene: TMPO
{'interaction_partners': 69, 'complex_participation': False}
Processing gene: RABGAP1L
{'interaction_partners': 51, 'complex_participation': False}
Processing gene: NUDT4
{'interaction_partners': 6, 'complex_participation': False}
Processing gene: GNAS
{'interaction_partners': 25, 'complex_participation': False}
Processing gene: MYL12A
{'interaction_partners': 41, 'complex_participation': False}
Processing gene: MOCS2
{'interaction_partners': 11, 'complex_participation': False}
Processing gene: TOR1AIP2
{'interaction_partners': 86, 'complex_participation': False}


### 오류 발생 유전자 데이터 재수집

In [ ]:
no_uniprot_id = list(no_uniprot_id)
intact_data_retrieval_failed = list(intact_data_retrieval_failed)
multi_uniprot_id_genes = list(multi_uniprot_id_genes)

In [ ]:
# Merge the three lists and remove duplicates
error_genes = list(set(no_uniprot_id + intact_data_retrieval_failed + multi_uniprot_id_genes))
len(error_gene_names)

67

In [ ]:
# Initialize an empty list to store gene data
gene_data_list = []

for gene in error_genes:
    print(f"Processing gene: {gene}")
    gene_data = {'gene': gene}

    # UniProt
    uniprot_info = get_uniprot_info(gene)
    gene_data.update(uniprot_info)

    # STRING
    string_info = get_string_info(gene)
    gene_data.update(string_info)

    # ClinVar
    clinvar_info = get_clinvar_info(gene)
    gene_data.update(clinvar_info)

    # KEGG
    kegg_info = get_kegg_info(gene)
    gene_data.update(kegg_info)

    # Gene Ontology
    go_info = get_go_info(gene)
    gene_data.update(go_info)

    # IntAct
    intact_info = get_intact_info(gene)
    gene_data.update(intact_info)

    gene_data_list.append(gene_data)
    time.sleep(0.2)  # To avoid overwhelming the servers

Processing gene: AQP7
Using ID for search: O14520
IntAct data retrieval failed for gene AQP7: list index out of range. Trying alternative method...
Using ID for search: O14520
Processing gene: THEG


WARNING [bioservices.KEGG:596]:  status is not ok with Not Found
WARNING [bioservices.KEGG:1181]:  Could not parse the entry correctly.


Using ID for search: P0DJG4
IntAct data retrieval failed for gene THEG: list index out of range. Trying alternative method...
Using ID for search: P0DJG4
Processing gene: NUDT4
Using ID for search: Q9NZJ9
Processing gene: CLN8
Using ID for search: Q9UBY8
IntAct data retrieval failed for gene CLN8: list index out of range. Trying alternative method...
Using ID for search: Q9UBY8
Processing gene: CLPS
Using ID for search: P04118
IntAct data retrieval failed for gene CLPS: list index out of range. Trying alternative method...
Using ID for search: P04118
Processing gene: PAOX
Using ID for search: Q6QHF9
IntAct data retrieval failed for gene PAOX: list index out of range. Trying alternative method...
Using ID for search: Q6QHF9
Processing gene: CXCL10
Using ID for search: P02778
IntAct data retrieval failed for gene CXCL10: list index out of range. Trying alternative method...
Using ID for search: P02778
Processing gene: PRPF4B
No UniProt ID found for gene: PRPF4B


WARNING [bioservices.KEGG:596]:  status is not ok with Not Found
WARNING [bioservices.KEGG:1181]:  Could not parse the entry correctly.


No UniProt ID found for gene: PRPF4B. Trying RefSeq Protein ID...
No RefSeq Protein ID found for gene: PRPF4B
Processing gene: CD40LG
Using ID for search: P29965
IntAct data retrieval failed for gene CD40LG: list index out of range. Trying alternative method...
Using ID for search: P29965
Processing gene: MT3
Using ID for search: P25713
IntAct data retrieval failed for gene MT3: list index out of range. Trying alternative method...
Using ID for search: P25713
Processing gene: SELL
Using ID for search: P14151
IntAct data retrieval failed for gene SELL: list index out of range. Trying alternative method...
Using ID for search: P14151
Processing gene: PAX4
No UniProt ID found for gene: PAX4
No UniProt ID found for gene: PAX4. Trying RefSeq Protein ID...
Using ID for search: NP_001353039.1
IntAct data retrieval failed for gene PAX4: list index out of range. Trying alternative method...
No UniProt ID found for gene: PAX4. Trying RefSeq Protein ID...
Using ID for search: NP_001353039.1
Proce

...

No UniProt ID found for gene: HBBP1. Trying RefSeq Protein ID...
No RefSeq Protein ID found for gene: HBBP1
Processing gene: CCR7
Using ID for search: P32248
IntAct data retrieval failed for gene CCR7: list index out of range. Trying alternative method...
Using ID for search: P32248
Processing gene: LCT
Using ID for search: P09848
IntAct data retrieval failed for gene LCT: list index out of range. Trying alternative method...
Using ID for search: P09848
Processing gene: SLC46A3
Using ID for search: Q7Z3Q1
IntAct data retrieval failed for gene SLC46A3: list index out of range. Trying alternative method...
Using ID for search: Q7Z3Q1
Processing gene: RHD
Using ID for search: Q02161
IntAct data retrieval failed for gene RHD: list index out of range. Trying alternative method...
Using ID for search: Q02161
Processing gene: CROCCP2
No UniProt ID found for gene: CROCCP2
STRING API call failed for gene CROCCP2: 400 Client Error: Bad Request for url: https://string-db.org/api/json/network?iden

WARNING [bioservices.KEGG:596]:  status is not ok with Not Found
WARNING [bioservices.KEGG:1181]:  Could not parse the entry correctly.


No UniProt ID found for gene: CROCCP2. Trying RefSeq Protein ID...
No RefSeq Protein ID found for gene: CROCCP2
Processing gene: CXCR6
Using ID for search: O00574
IntAct data retrieval failed for gene CXCR6: list index out of range. Trying alternative method...
Using ID for search: O00574
Processing gene: ITGBL1
Using ID for search: O95965
IntAct data retrieval failed for gene ITGBL1: list index out of range. Trying alternative method...
Using ID for search: O95965
Processing gene: ACKR1
Using ID for search: Q16570
IntAct data retrieval failed for gene ACKR1: list index out of range. Trying alternative method...
Using ID for search: Q16570
Processing gene: MOCS2
Using ID for search: O96007
Processing gene: RDH16
Using ID for search: O75452
IntAct data retrieval failed for gene RDH16: list index out of range. Trying alternative method...
Using ID for search: O75452
Processing gene: HTR1D
Using ID for search: P28221
IntAct data retrieval failed for gene HTR1D: list index out of range. Tr

WARNING [bioservices.KEGG:596]:  status is not ok with Not Found
WARNING [bioservices.KEGG:1181]:  Could not parse the entry correctly.


No UniProt ID found for gene: PTTG3P. Trying RefSeq Protein ID...
No RefSeq Protein ID found for gene: PTTG3P
Processing gene: SAC3D1
No UniProt ID found for gene: SAC3D1
No UniProt ID found for gene: SAC3D1. Trying RefSeq Protein ID...
Using ID for search: NP_001354414.1
IntAct data retrieval failed for gene SAC3D1: list index out of range. Trying alternative method...
No UniProt ID found for gene: SAC3D1. Trying RefSeq Protein ID...
Using ID for search: NP_001354414.1
Processing gene: TARP
STRING API call failed for gene TARP: 400 Client Error: Bad Request for url: https://string-db.org/api/json/network?identifiers=TARP&species=9606&required_score=400
Using ID for search: A2JGV3
IntAct data retrieval failed for gene TARP: list index out of range. Trying alternative method...
Using ID for search: A2JGV3
Processing gene: CAPN9
Using ID for search: O14815
IntAct data retrieval failed for gene CAPN9: list index out of range. Trying alternative method...
Using ID for search: O14815
Proces

In [ ]:
revised_gene_info_df = pd.DataFrame(gene_data_list)
revised_gene_info_df

,gene,sequence_length,sequence,interaction_count,highest_score,highest_score_partner,variant_count,has_pathogenic,pathway_count,main_pathway,molecular_function,biological_process,interaction_partners,complex_participation
0,AQP7,342,MVQASGHRRSTRGSKMVSWSVIAKIQEILQRKMVREFLAEFMSTYV...,42,0.958,GK2,82,True,2,PPAR signaling pathway,"[protein binding, urea transmembrane transport...","[water transport, urea transmembrane transport...",8,False
1,THEG,465,MENQEFLSSSAPSEVTDGQVSTEISTCSEVFQKPIVLRILDTHREL...,14,0.965,CCT5,11,True,0,Unknown,[Unknown],[Unknown],0,False
2,NUDT4,180,MMKFKPNQTRTYDREGFKKRAACLCFRSEQEDEVLLVSSSRYPDQW...,47,0.836,NUDT15,11,True,0,Unknown,[5'-(N(7)-methylguanosine 5'-triphospho)-[mRNA...,"[diadenosine pentaphosphate catabolic process,...",6,False
3,CLN8,286,MNPASDGGTSESIFDLDYASWGIRSTLMVAGFVFYLGVFVVCHQLS...,47,0.997,CLN3,704,True,0,Unknown,"[protein binding, ceramide binding]","[mitochondrial membrane organization, nervous ...",5,False
4,CLPS,112,MEKILILLLVALSVAYAAPGPRGIIINLENGELCMNSAQCKSNCCQ...,35,0.999,PNLIP,14,True,1,Fat digestion and absorption,"[enzyme activator activity, lipase binding]","[lipid metabolic process, response to bacteriu...",0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,G6PC2,355,MDFLHRNGVLIIQHLQKDYRAYYTFLNFMSNVGDPRNIFFIYFPLC...,52,0.999,G6PD,39,True,12,Glycolysis / Gluconeogenesis,[glucose-6-phosphatase activity],"[gluconeogenesis, regulation of insulin secret...",0,False
63,ITIH5,942,MLLLLGLCLGLSLCVGSQEEAQSWGHSSEQDGLRVPRQVRLLQRLK...,4,0.700,PRELP,124,True,0,Unknown,[serine-type endopeptidase inhibitor activity],"[hyaluronan metabolic process, negative regula...",10,False
64,GPR19,415,MVFAHRMDNSKPHLIIPTLLVPLQNRSCTETATPLPSQYLMELSEE...,14,0.903,ENHO,72,True,0,Unknown,[G protein-coupled receptor activity],[G protein-coupled receptor signaling pathway],0,False
65,TEX15,0,,27,0.979,DMC1,363,True,0,Unknown,[Unknown],[Unknown],0,False


In [ ]:
gene_info_df = pd.read_csv('gene_info_error.csv')
gene_info_df.head()

,gene,sequence_length,sequence,interaction_count,highest_score,highest_score_partner,variant_count,has_pathogenic,pathway_count,main_pathway,molecular_function,biological_process,interaction_partners,complex_participation
0,TANK,425,MDKNIGEQLNKAYEAFRQACMDRDSAVKELQQKTENYEQRIREQQE...,52,0.999,TNFRSF1A,51,True,6,Autophagy - animal,"['cysteine-type deubiquitinase activity', 'ubi...",['negative regulation of tumor necrosis factor...,216,False
1,PPP1CA,330,MSDSEKLNLDSIIGRLLEVQGSRPGKNVQLTENEIRGLCLKSREIF...,34,0.999,PPP1R15A,29,True,22,mRNA surveillance pathway,"['protein binding', 'phosphoprotein phosphatas...",['protein dephosphorylation'],306,False
2,PRC1,620,MRRSEVLAEESIVCLQKALNHLREIWELIGIPEDQRLQRTEVVKKH...,55,0.999,KIF23,105,True,0,Unknown,"['protein binding', 'microtubule binding', 'ki...","['regulation of cytokinesis', 'microtubule cyt...",32,False
3,NDUFAF4,175,MGALVIRGIRNFNLENRAEREISKMKPSVAPRHPSTNSLLREQISL...,55,0.999,NDUFS7,138,True,1,Thermogenesis,"['calmodulin binding', 'protein binding']",['mitochondrial respiratory chain complex I as...,76,False
4,PGM3,542,MDLGAITKYSALHAKPNGLILQYGTAGFRTKAEHLDHVMFRMGLLA...,35,0.997,GNPNAT1,568,True,3,Amino sugar and nucleotide sugar metabolism,"['phosphoacetylglucosamine mutase activity', '...","['hemopoiesis', 'spermatogenesis', 'carbohydra...",23,False


In [ ]:
# 'gene' 열을 기준으로 오류 데이터 병합 (update)
gene_info_df.set_index('gene', inplace=True)
revised_gene_info_df.set_index('gene', inplace=True)

# 오류 유전자의 정보를 업데이트
gene_info_df.update(revised_gene_info_df)

# 인덱스 재설정
gene_info_df.reset_index(inplace=True)

In [ ]:
# Save to CSV for future use
gene_info_df.to_csv('gene_info.csv', index=False)

# 최종 데이터 확인

In [ ]:
import pandas as pd
gene_info_df = pd.read_csv('gene_info.csv')
gene_info_df.head()

,gene,sequence_length,sequence,interaction_count,highest_score,highest_score_partner,variant_count,has_pathogenic,pathway_count,main_pathway,molecular_function,biological_process,interaction_partners,complex_participation
0,TANK,425,MDKNIGEQLNKAYEAFRQACMDRDSAVKELQQKTENYEQRIREQQE...,52,0.999,TNFRSF1A,51,True,6,Autophagy - animal,"['cysteine-type deubiquitinase activity', 'ubi...",['negative regulation of tumor necrosis factor...,216,False
1,PPP1CA,330,MSDSEKLNLDSIIGRLLEVQGSRPGKNVQLTENEIRGLCLKSREIF...,34,0.999,PPP1R15A,29,True,22,mRNA surveillance pathway,"['protein binding', 'phosphoprotein phosphatas...",['protein dephosphorylation'],306,False
2,PRC1,620,MRRSEVLAEESIVCLQKALNHLREIWELIGIPEDQRLQRTEVVKKH...,55,0.999,KIF23,105,True,0,Unknown,"['protein binding', 'microtubule binding', 'ki...","['regulation of cytokinesis', 'microtubule cyt...",32,False
3,NDUFAF4,175,MGALVIRGIRNFNLENRAEREISKMKPSVAPRHPSTNSLLREQISL...,55,0.999,NDUFS7,138,True,1,Thermogenesis,"['calmodulin binding', 'protein binding']",['mitochondrial respiratory chain complex I as...,76,False
4,PGM3,542,MDLGAITKYSALHAKPNGLILQYGTAGFRTKAEHLDHVMFRMGLLA...,35,0.997,GNPNAT1,568,True,3,Amino sugar and nucleotide sugar metabolism,"['phosphoacetylglucosamine mutase activity', '...","['hemopoiesis', 'spermatogenesis', 'carbohydra...",23,False


In [ ]:
gene_info_df.sequence_length.argmax()

np.int64(769)

In [ ]:
gene_info_df.iloc[[769],:]

,gene,sequence_length,sequence,interaction_count,highest_score,highest_score_partner,variant_count,has_pathogenic,pathway_count,main_pathway,molecular_function,biological_process,interaction_partners,complex_participation
769,SYNE1,8797,MATSRGASRCPRDIANVMQRLQDEQEIVQKRTFTKWINSHLAKRKP...,45,0.999,SYNE2,6244,True,1,Cytoskeleton in muscle cells,"['signaling receptor binding', 'lamin binding'...",['negative regulation of mesenchymal cell apop...,49,False


# 확장 데이터 추가

In [ ]:
# Load gene information
gene_info_df = pd.read_csv('gene_info.csv')

# Transpose the mutation data to have genes as rows
mutation_data = train_df.set_index(['Sample_ID', 'SUBCLASS']).T

# Reset index to have 'gene' as a column
mutation_data.reset_index(inplace=True)
mutation_data.rename(columns={'index': 'gene'}, inplace=True)

# Merge mutation data with gene information
merged_data = pd.merge(gene_info_df, mutation_data, on='gene', how='inner')

In [ ]:
merged_data.to_csv('expended_train.csv', index=False)